ABSA utilizando el modelo DistilBert

In [9]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
from tqdm import tqdm

# Paso 1: Cargar datos preprocesados
data_path = "sample_preprocessed.parquet"
data = pd.read_parquet(data_path)

# Paso 2: Configurar el modelo y tokenizer (DistilBERT fine-tuned para clasificación)
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Optimización para CPU
device = torch.device("cpu")  # Asegura que use CPU
model.to(device)

# Crear pipeline con truncamiento automático de textos largos
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    device=-1,  # -1 indica que se usará CPU
    truncation=True,  # Trunca automáticamente los textos largos
    max_length=512  # Límite para DistilBERT
)

# Paso 3: Realizar ABSA por lotes
def analyze_sentiment_batch(data_batch, aspects):
    """
    Analiza un lote de datos, evaluando el sentimiento para cada aspecto.
    """
    results = []
    for _, row in data_batch.iterrows():
        text = row["text_clean"]
        row_results = []
        for aspect in aspects:
            input_text = f"What do people think about {aspect}? {text}"
            sentiment = sentiment_pipeline(input_text)
            row_results.append({
                "aspect": aspect,
                "sentiment": sentiment[0]["label"],
                "score": sentiment[0]["score"]
            })
        results.append(row_results)
    return results

# Definir aspectos
aspects = ["service", "food", "price", "ambiance"]

# Dividir los datos en lotes y procesarlos
batch_size = 50  # Ajusta según la capacidad de tu CPU
all_results = []
for i in tqdm(range(0, len(data), batch_size)):
    batch = data.iloc[i:i+batch_size]
    batch_results = analyze_sentiment_batch(batch, aspects)
    all_results.extend(batch_results)

# Agregar los resultados al DataFrame
data["aspect_sentiments"] = all_results

# Paso 4: Guardar los resultados
output_path = "absa_results_optimized.parquet"
data.to_parquet(output_path, index=False)
print(f"Análisis completado y guardado en {output_path}")


100%|██████████| 60/60 [30:09<00:00, 30.16s/it]

Análisis completado y guardado en absa_results_optimized.parquet


In [10]:
# Paso 4: Guardar los resultados en CSV
output_path_csv = "absa_results_optimized.csv"
data.to_csv(output_path_csv, index=False)
print(f"Análisis completado y guardado en {output_path_csv}")


Análisis completado y guardado en absa_results_optimized.csv


In [11]:
import pandas as pd

# Paso 1: Cargar el archivo CSV generado anteriormente
input_csv_path = "absa_results_optimized.csv"
data = pd.read_csv(input_csv_path)

# Paso 2: Filtrar las columnas deseadas
columns_to_keep = ["id", "user_id", "stars", "date", "id_business", "text_clean", "sentiment", "aspect_sentiments"]
filtered_data = data[columns_to_keep]

# Paso 3: Guardar el nuevo archivo CSV
output_csv_path = "filtered_absa_results.csv"
filtered_data.to_csv(output_csv_path, index=False)

print(f"Archivo filtrado creado y guardado en {output_csv_path}")


Archivo filtrado creado y guardado en filtered_absa_results.csv


In [12]:
import pandas as pd

# Paso 1: Cargar el archivo CSV generado anteriormente
input_csv_path = "absa_results_optimized.csv"
data = pd.read_csv(input_csv_path)

# Paso 2: Filtrar las columnas deseadas
columns_to_keep = ["stars", "date", "id_business", "sentiment", "aspect_sentiments"]
filtered_data = data[columns_to_keep]

# Paso 3: Guardar el nuevo archivo CSV
output_csv_path = "filtered_absa_results_2.csv"
filtered_data.to_csv(output_csv_path, index=False)

print(f"Archivo filtrado creado y guardado en {output_csv_path}")


Archivo filtrado creado y guardado en filtered_absa_results_2.csv
